<img src="imgs/frameworks.png">
http://www.youtube.com/watch?v=AYjPIMe0BhA

<img src="imgs/flask.png">

In [4]:
%%writefile fhello.py
from flask import Flask
app = Flask(__name__)

@app.route("/")
def hello():
    return "Hello World, for real!"

if __name__ == "__main__":
    app.run()

Overwriting fhello.py


In [5]:
%run fhello.py 

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [09/Apr/2018 15:00:57] "GET / HTTP/1.1" 200 -


### URL Route Registration

Where users can go to get things on your site
using “view decorators” of “view functions”

```python

@app.route('/')
....
@app.route('/hello')
...
@app.route('/user/<username>')
def show_user_profile(username):
    return 'User %s' % username

@app.route('/post/<int:post_id>')
def show_post(post_id):
    return 'Post %d' % post_id
```

In [ ]:
# %load urls.py
from flask import Flask

app = Flask(__name__)
app.debug = True

## this will route / and /hello 
## the function "hi()" is called a "view function"
@app.route("/")
@app.route('/hello')
def hi():
    return "<font color='red'>Hello!</font>"

## this will route URLs like: /user/josh 
@app.route('/user/<username>')
def show_user_profile(username):
    return 'hello, username =  %s' % username

## this will route URLs like: /post/1234
@app.route('/post/<int:post_id>')
def show_post(post_id):
    return 'Post # = %d' % post_id

# here we show off multiple input and defaults
@app.route("/doc/<int:docid>/page/<int:pageid>")
@app.route("/doc/<int:docid>", defaults={'pageid': 10})
def show_document_pages(docid,pageid):
    return "Doc = %i  and Page = %i" % (docid,pageid)

## a different way to add URL rules
## this connects the function hi() to the url /hola
## nice thing by doing it this way is that you could see all your
## mappings in one place
app.add_url_rule('/hola', "say_hola", hi)

if __name__ == "__main__":
    app.run()

In [7]:
%run urls.py

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


### HTTP Methods

```python
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        do_the_login()
    else:
        show_the_login_form()
```

In [13]:
!cat methods.py

from flask import Flask, redirect, request, url_for
from flask import current_app

app = Flask("Dan")
app.debug = True

def debug():
    assert current_app.debug == False, "Don't panic! You're here by request of debug()"

## we can tell our view functions what HTTP methods it
## is allowed to respond to
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        return "performing the log in"
    else:
    	## this is a normal GET request
        # debug()
        return "please log in..."

@app.route("/")
@app.route("/index.html")
def redirect_to_login():
	## 301 is an HTTP error code
	return redirect("/login",301)

#app.run()
if __name__ == "__main__":
    app.run()


### Forms

but it’s annoying to have to put HTML into Python (see `form1.py` file)...

```python
@app.route('/welcome', methods=['GET', 'POST'])
def welcomehi():

    if request.method == 'POST':
	    username = request.form['name']
	    if username not in (""," ",None):
	    	return "Hey %s, what's up?" % username
	    else:
	    	return """We really want to know your name. Add it 
	    	          <a href='%s'>here</a>""" % url_for("welcomehi")
    else:
    	## this is a normal GET request
        return '''
            <form action="welcome" method="POST">
            What is your name?
            <input type="text" name="name" />
            <input type="submit" />
            </form>''
```

In [ ]:
# %load form1.py
from flask import Flask, redirect, request, url_for

app = Flask(__name__)
app.debug = True

## we can tell our view functions what HTTP methods it
## is allowed to respond to
@app.route('/welcome', methods=['GET', 'POST'])
def welcomehi():

    if request.method == 'POST':
	    username = request.form['name']
	    if username not in (""," ",None):
	    	return "Hey %s, what's up?" % username
	    else:
	    	return """We really want to know your name. Add it 
	    	          <a href='%s'>here</a>""" % url_for("welcomehi")
    else:
    	## this is a normal GET request
        return '''
            <form action="welcome" method="POST">
            What is your name?
            <input type="text" name="name" />
            <input type="submit" />
            </form>'''

@app.route("/")
def redirect_to_login():
	## 301 is an HTTP error code
	return redirect(url_for("welcomehi"),301)

if __name__ == "__main__":
    app.run()

## Templates

What users see. You need to know HTML to make these.

`templates/base.html`
```html
<!DOCTYPE html>
<html>
    <head>
        <title>My Super Site</title>
    </head>
    
    <body>
        <h1>{{ page_title }}</h1>

        <p>This content is “dynamic”:</p>
{% block content %}{% endblock %}        
    </body>
</html>
```

In [ ]:
%%writefile thello.py

from flask import Flask, render_template, request, url_for
app = Flask(__name__)
app.debug = True

@app.route("/test")
def hello():
    return render_template('base.html', 
                            page_title="Templates", 
                            content="hello!")

@app.route('/welcome', methods=['GET', 'POST'])
def welcome():

    if request.method == 'POST':
        username = request.form['name']
        if username not in (""," ",None):
            #return "Hey %s, what's up?" % username
            return render_template('base.html', 
                                   page_title="Hey %s, what's up?" % username, 
                                   content="Hey %s, what's up?" % username)
        else:
            return """We really want to know your name. Add it 
                     <a href='%s'>here</a>""" % url_for("welcome")
    else:
        ## this is a normal GET request
        return render_template("form.html")

if __name__ == "__main__":
    app.run()

`templates/form.html`:
```html
{% extends "base.html" %}

{% set page_title = 'My Form' %}

{% block content %}
            <form action="welcome" method="POST">
            What is your name?
            <input type="text" name="name" />
            <input type="submit" />
            </form>
{% endblock %}
```

(flask uses Jinja2 templating: http://jinja.pocoo.org/docs/2.10/templates/)

In [16]:
%run thello.py

 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
 * Restarting with stat


## Flask with  MVC-like behavior

<img src="https://files.realpython.com/media/mvc_diagram_with_routes.e12c5b982ac8.png">

https://realpython.com/blog/python/the-model-view-controller-mvc-paradigm-summarized-with-legos/

Flask can behave something like it with SQLAlchemy...
`pip install flask-sqlalchemy`

### Models 
The classes that “model” the data objects that make up your app. Stored in whatever database your config sets.

```python
from flask import Flask
from flask.ext.sqlalchemy import SQLAlchemy

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:////tmp/test.db'
db = SQLAlchemy(app)


class User(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(80), unique=True)
    email = db.Column(db.String(120), unique=True)

    def __init__(self, username, email):
        self.username = username
        self.email = email
```

<img src="imgs/model.png">

## Restful apps

Flask-Restless provides simple generation of ReSTful APIs for database models defined using SQLAlchemy (or Flask-SQLAlchemy). The generated APIs send and receive messages in JSON format.

http://flask-restless.readthedocs.io/en/stable/

In [ ]:
!pip install flask-restless
!pip install flask-sqlalchemy

In [23]:
%%writefile rest.py
import flask
import flask_sqlalchemy
import flask_restless

app = flask.Flask(__name__)
app.config["DEBUG"] = True
app.config["SQLALCHEMY_DATABASE_URI"] = "sqlite:////tmp/test_new.db"
app.config["SQLALCHEMY_TRACK_MODIFICATIONS"] = False

db = flask_sqlalchemy.SQLAlchemy(app)

class Member(db.Model):
        # __tablename __ = "newsletter_members"
        id = db.Column(db.Integer,primary_key=True)
        last_name = db.Column(db.String(50))
        first_name = db.Column(db.String(120))
        email = db.Column(db.String(120),unique=True)

db.create_all()

manager = flask_restless.APIManager(app,flask_sqlalchemy_db=db)

manager.create_api(Member,methods=["GET","POST"])

app.run()

Overwriting rest.py


Now run it in the terminal...

In [44]:
import json  # import simplejson as json, if on Python 2.5
import requests  # python-requests is installable from PyPI...
# newmember = {'last_name': 'Clinton', 'first_name': 'Hillary', 'email': 'hillary2000@yahoo.com'}

newmember = {'last_name': 'Trump', 'first_name': 'Donald', 'email': 'prez_never@yahoo.com'}
r = requests.post('http://127.0.0.1:5000/api/member', data=json.dumps(newmember),
                   headers={'content-type': 'application/json'})

In [45]:
r.status_code

201

In [46]:
r.headers['content-type']

'application/json, application/json'

In [47]:
r.json()

{'email': 'prez_never@yahoo.com',
 'first_name': 'Donald',
 'id': 3,
 'last_name': 'Trump'}

In [48]:
newid = json.loads(r.text)['id']
newid = 2

In [49]:
r = requests.get('http://127.0.0.1:5000/api/member/%s' % newid,
                  headers={'content-type': 'application/json'})

In [50]:
r.status_code, r.headers['content-type']

(200, 'application/json, application/json')

In [51]:
r.json()

{'email': 'prez_forever@yahoo.com',
 'first_name': 'Donald',
 'id': 2,
 'last_name': 'Trump'}

## Flask Apps


The philosophy of modern web frameworks is “Don’t Repeat Yourself” (DRY).

Flask is already good at supplying the DRY building blocks for low-level tasks, but what about high-level functionality?

- User registration (Flask-Login)

- Sending Mail (Flask-Mail)

The Flask community makes reusable apps to solve this problem. Plug it in and go.

http://flask.pocoo.org/extensions/